# Prerequisites and Installation

## Virtual Environment

Ensure to have PIP installed. On Ubuntu:

```sh
apt install python3-pip
```

Create your virtual environment

```sh
python3 -m venv ./venv
```

Then once it's created, you need to initialise it before setting the Jupyter
Kernel:

```sh
. ./venv/bin/activate
```

Choose the `venv` Python kernel from within VSCode. Follow instructions for
installing the `ipykernel` which is needed to run Jupiter. It's installed safely
in your virtual environment.

If you can't see the `venv` as the kernel in VSCode, try changing to the
directory where the `venv` folder is (not the `venv` folder itself, but one
before it) and starting VSCode from there. Else you may unintentionally install
all the packages needed in your global or user configuration.

## Ubuntu Packages

Install the following packages.

- Allows exporting Jupyter notebooks as PDF:

  ```sh
  apt install texlive-xetex
  ```

## Python Tools

Install the `matplotlib`, which will also install `numpy`.

In [ ]:
%pip install matplotlib


# Capturing Data

You'll need to run the benchmark tests to capture the data. These files are
stored in `.json` format.

Under linux, Google Benchmark recommends to set the CPU governor to performance
to avoid variations in running the benchmarks.

```sh
sudo cpupower frequency-set --governor performance
```

Under Linux, there is the script `run_linux.sh` which runs with various
`mallopt()` options.

## Running Benchmarks Manually

Under Linux, you will want to run a command similar to the following:

```sh
export BASE_NAME=mymachine
taskset -c 1 ./benchmarks/malloc/malloc_bench -mM_MMAP_THRESHOLD=0 --benchmark_out_format=json --benchmark_out=${BASE_NAME}_mmap.json
```

And further, you'd likely want to run multiple times with various different
options for the `mallopt()` (by setting `-m` or setting environment variables as
described by your system documentation).

# Processing Data

## Configuration File for What to Process

Create a text file that is a collection of key/value pairs. Update the
configuration file variable `CONFIG_FILE` to load what you need.

This is a `.json` file that is a simple dictionary of a data-set name, and the
google-benchmark `.json` output file with the test results.

For example, [`i9-13950hx_Linux.json`](./i9-13950hx_Linux.json) contains a small
JSON snippet, which is a dictionary of the data sets (the key is the name of the
dataset that will be plotted later), and the file captured from the previous
step.

In [2]:
CONFIG_FILE = "rpi5_linux.json"

In [3]:
import json
import os

if not os.path.isfile(CONFIG_FILE):
    raise Exception(f"File '{CONFIG_FILE}' not found")

with open(CONFIG_FILE, encoding="utf-8") as config_file:
    config_file_results = json.load(config_file)

data = {}
base_results_path = os.path.dirname(os.path.abspath(CONFIG_FILE))
for test_run in config_file_results:
    google_test_file = os.path.relpath(os.path.join(base_results_path, config_file_results[test_run]))
    if os.path.exists(google_test_file):
        if test_run in data:
            raise Exception(f"Test run {test_run} defined multiple times in {CONFIG_FILE}")
        data[test_run] = {}
        data[test_run]["file"] = google_test_file
        data[test_run]["benchmark"] = {}
        data[test_run]["results"] = {}
    else:
        raise Exception(f"Path {google_test_file} not found")

# Now the `data` variable can be enumerated by its keys to get the dataruns. We
# don't need CONFIG_FILE any longer.
#del CONFIG_FILE


## Internal Representation of Benchmark and Result Data in Python

The data is organised in Python as a single variable. Values in angled quotes
`<xx>` indicate a key name. It might be a string or an integer depending on the
context. Note, some of the variables haven't been discussed yet, but the
complete structure is defined here for a single reference when reading code.

```json
{
    "<test_run>":  {
        "file": "<relative path>",
        "benchmark": {
            "<name>": {
                "<size>": {
                    "real_time": "<nanoseconds>"
                }
            },
        },
        "results": {
            "<size>": {
                "t_Tm": "<value>",   // Total Malloc/Free
                "t_m": "<value>",    // Malloc
                "t_fm": "<value>",   // Free
                "t_Tc": "<value>",   // Total Calloc/Free
                "t_c": "<value>",    // Calloc
                "t_fc": "<value>",   // Free
                "t_wm": "<value>",   // Walk Time (Malloc/Free)
                "t_wc": "<value>"    // Walk Time (Malloc/MemSet/Free)
            }
        }
    }
}
```

Each "`<test_run>`" is loaded from a `.json` file referenced by the
`CONFIG_FILE`.


In [4]:
import re

for test_run in data.keys():
    with open(data[test_run]["file"], encoding="utf-8") as google_result:
        google_data = json.load(google_result)

    # Parse through the file, extracting the contents and putting it into the
    # new dictionary.
    for benchmark in google_data["benchmarks"]:
        match = re.search("BM_(\S+)/(\d+)", benchmark["name"])
        if match is not None:
            malloc_name = match.group(1)
            malloc_size = int(match.group(2))
            realtime = benchmark["real_time"]

            if malloc_name not in data[test_run]["benchmark"]:
                data[test_run]["benchmark"][malloc_name] = {}
            if malloc_size not in data[test_run]["benchmark"][malloc_name]:
                data[test_run]["benchmark"][malloc_name][malloc_size] = {}
            data[test_run]["benchmark"][malloc_name][malloc_size]["real_time"] = realtime

#print(json.dumps(data, indent=4))

## Plotting Times for `malloc()`/clear/`free()`

We calculate the data for each "`<test_run>`", and they're independent of each
other for this type of graph. The "`<name>`" is the name of the benchmark, and
these benchmarks are referred to each other within a test to estimate metrics.

### Calculating the Time for `malloc()` and `free()` (and `calloc()`)

Google Benchmark runs by executing contents in a loop multiple times, and then
estimating the average time for each loop run. This means that a `malloc()` may
run a large number of times, such that if run too often, the benchmark will
crash due to out of memory. So the loop must necessarily be followed by a
`free()` call that the loop can be run an arbitrary number of times.

Hence the test `BM_MallocFreeBenchmark` executes a `malloc()` followed by a
`free()` and the measurement is the average time of _both_.

To estimate the time required in just the `malloc()` call and just the `free()`
call, we must run benchmarks that use the Google Benchmark function
`PauseTiming()` and `ResumeTiming()`. This introduces overhead, such that:

Let:

\begin{align}
T_{MallocBench} &= T_M + T_P
\end{align}

and:

\begin{align}
T_{FreeBench} &= T_F + T_P
\end{align}

So that $T_M$ is the actual time for the `malloc()` call, and $T_F$ is the time
for the `free()` call. The overhead $T_P$ is due to the `PauseTiming()` and
`ResumeTiming()` overhead, which for very fast function calls (of the order of
up to 1000ns for $T_M$ or $T_F$ makes a reasonable influence).

Then we know we've measured:

\begin{align}
T_{MallocFreeBench} &= T_M + T_F \\
T_{MallocFreeBench} &= T_{MallocBench} + T_{FreeBench} + 2 T_P
\end{align}

So given all three measurements, we can calculate what the overhead is, and
calculate the true value of $T_M$ and $T_F$ which is what we want to plot:

\begin{align}
T_P &= \frac{1}{2} \left( T_{MallocFreeBench} - T_{MallocBench} - T_{FreeBench} \right)
\end{align}

so that:

\begin{align}
T_M &= \frac{1}{2} \left( T_{MallocFreeBench} + T_{MallocBench} - T_{FreeBench} \right) \\
T_F &= \frac{1}{2} \left( T_{MallocFreeBench} - T_{MallocBench} + T_{FreeBench} \right)
\end{align}

Similarly, the same equations can be used for the `calloc()` call to estimate the sizes.

#### Code to Calculate from the Formula

In [5]:
for test_run in data:
    # Get the sizes used for each benchmark. They should be consistent.
    sizes = []
    for benchmark in data[test_run]["benchmark"]:
        for size in data[test_run]["benchmark"][benchmark]:
            if size not in sizes:
                sizes.append(size)

    for size in sizes:
        mallocfree_time = data[test_run]["benchmark"]["MallocFreeBench"][size]["real_time"]
        malloc_time = data[test_run]["benchmark"]["MallocBench"][size]["real_time"]
        mfree_time = data[test_run]["benchmark"]["MFreeBench"][size]["real_time"]
        t_m = 0.5 * (mallocfree_time + malloc_time - mfree_time)
        t_fm = 0.5 * (mallocfree_time - malloc_time + mfree_time)

        callocfree_time = data[test_run]["benchmark"]["CallocFreeBench"][size]["real_time"]
        calloc_time = data[test_run]["benchmark"]["CallocBench"][size]["real_time"]
        cfree_time = data[test_run]["benchmark"]["CFreeBench"][size]["real_time"]
        t_c = 0.5 * (callocfree_time + calloc_time - cfree_time)
        t_fc = 0.5 * (callocfree_time - calloc_time + cfree_time)

        mallocwalk_time = data[test_run]["benchmark"]["MallocWalkFreeBench"][size]["real_time"]
        t_wm = mallocwalk_time - mallocfree_time

        mallocset_time = data[test_run]["benchmark"]["MallocClearFreeBench"][size]["real_time"]
        mallocsetwalk_time = data[test_run]["benchmark"]["MallocClearWalkFreeBench"][size]["real_time"]
        t_wc = mallocsetwalk_time - mallocset_time

        if size not in data[test_run]["results"]:
            data[test_run]["results"][size] = {}
        data[test_run]["results"][size]["t_Tm"] = mallocfree_time
        data[test_run]["results"][size]["t_m"] = t_m
        data[test_run]["results"][size]["t_fm"] = t_fm
        data[test_run]["results"][size]["t_Tc"] = callocfree_time
        data[test_run]["results"][size]["t_c"] = t_c
        data[test_run]["results"][size]["t_fc"] = t_fc
        data[test_run]["results"][size]["t_wm"] = t_wm
        data[test_run]["results"][size]["t_wc"] = t_wc

#print(json.dumps(data, indent=4))

### Plotting Histograms of `malloc()` and `free()`

Now we can collect the data, process it so it can be graphed. `MatPlotLib` is
used to prepare the axis and draw the histograms that can be exported.

#### Preamble Code for Setting up the Plots

In [6]:
def unit_size(unit):
    match unit:
        case 0:
            return ""
        case 1:
            return "k"
        case 2:
            return "M"
        case 3:
            return "G"
        case _:
            return "?"

def auto_size(size):
    unit=0
    while size >= 1024:
        unit += 1
        size = size / 1024

    return f"{int(size)} {unit_size(unit)}B"

In [7]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

############################################################
# Prepare the data to be easily plotted
# - strsizes are the labels. It's not scaled on the x-axis.
# - malloc_perf is the "total" time
# - malloc_perf_m is the time to do just the alloc.

sizes = []
for test_run in data:
    # Get the sizes used for each benchmark. They should be consistent.
    for benchmark in data[test_run]["benchmark"]:
        for size in data[test_run]["benchmark"][benchmark]:
            if size not in sizes:
                sizes.append(size)
sizes = sorted(sizes)

strsizes = []
for size in sizes:
    #strsizes.append(str(int(size / 1024)))
    strsizes.append(auto_size(size))
strsizes = np.array(strsizes)

############################################################
# Graph the data

set_colors = [
    "#1f78b4", "#33a02c", "#e31a1c", "#ff7f00",
    "#6a3d9a", "#b15928", "#a6cee3", "#b2df8a",
    "#fb9a99", "#fdbf6f", "#cab2d6", "#ffff99",
]

def plot_malloc_benchmarks(title, strsizes, perf_malloc_free, perf_malloc, time_log_scale):
    x = np.arange(len(strsizes))   # Label locations
    width = 1 / (len(perf_malloc_free) + 1)    # We have "test_runs" bars per size

    fig, ax = plt.subplots(facecolor="lightgrey", figsize=(8,4), dpi=300)

    # The first plot is the total time for the operation.
    multiplier = 0
    for attribute, measurement in perf_malloc_free.items():
        offset = width * multiplier
        # - x is an array [0, 1, 2, ..., len(sizes)-1 ]
        # - offset shifts the plot exactly by one bar width, so each dataset appears
        #   side by side, looking like a histogram.
        rects = ax.bar(x + offset, measurement, width, label=attribute, color=set_colors[multiplier])
        multiplier += 1

    # The second plot overlays the "malloc" time with an alpha that makes it a bit
    # darker, se we can see how much time it takes to do an allocation.

    if perf_malloc is not None:
        multiplier = 0
        for attribute, measurement in perf_malloc.items():
            offset = width * multiplier
            # - x is an array [0, 1, 2, ..., len(sizes)-1 ]
            # - offset shifts the plot exactly by one bar width, so each dataset appears
            #   side by side, looking like a histogram.
            rects = ax.bar(x + offset, measurement, width, color="#001020", alpha=0.5)
            multiplier += 1

    plt.rc("legend", fontsize=5)
    ax.grid(visible=True, which="major", axis="both")
    ax.grid(visible=True, which="minor", axis="y")
    ax.set_title(f"{title} - {CONFIG_FILE}")
    ax.set_xlabel("Size")
    ax.set_xticks(x + (1-width)/2, strsizes)          # `(1-width)/2` aligns to the centre
    ax.set_xticklabels(strsizes, rotation=90)
    ax.tick_params(axis="both", which="major", grid_color="lightgrey", grid_linewidth=0.5, grid_linestyle="dashed")
    ax.tick_params(axis="y", which="minor", grid_color="lightgrey", grid_linewidth=0.5, grid_linestyle="dashed")
    if time_log_scale:
        ax.set_ylabel("Log_Time (µs)\n(Darker is 'alloc' time)")
        ax.set_yscale("log")
        ax.minorticks_on()
        ax.yaxis.set_major_locator(ticker.LogLocator(numticks=20))
        ax.yaxis.get_minor_locator().set_params(numticks=20, subs=[.2, .4, .6, .8])
    else:
        ax.set_ylabel("Time (µs)\n(Darker is 'alloc' time)")
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(10))
        ax.tick_params(axis="y", which="minor", grid_color="lightgrey", grid_linewidth=0.5, grid_linestyle="dashed")
    ax.set_axisbelow(True)
    ax.legend(loc="upper left")  # ncols=len(data)
    return ax

#### Plot of `malloc()` Speed

The `malloc()` function call returns data that is uninitialised.

In [ ]:
# Key is the test run, the value is an array of results in the order by sizes.
malloc_perf = {}
for test_run in data:
    malloc_perf[test_run] = []
    for size in sizes:
        malloc_perf[test_run].append(data[test_run]["results"][size]["t_Tm"] / 1000)
malloc_perf_m = {}
for test_run in data:
    malloc_perf_m[test_run] = []
    for size in sizes:
        malloc_perf_m[test_run].append(data[test_run]["results"][size]["t_m"] / 1000)

LOG=True
if not LOG:
    plot_malloc_benchmarks("malloc() speed", strsizes, malloc_perf, malloc_perf_m, LOG)
else:
    # WARNING: Plotting the two graphs on a logarithmic scale will cause
    # observers to read incorrectly the distorted data. If the `malloc()` and
    # the `free()` are the same amount of time, the user will think that the
    # `malloc()` (the darker graph) is significantly more time (which it isn't).
    plot_malloc_benchmarks("malloc()/free() speed (Logarithmic Scale)", strsizes, malloc_perf, malloc_perf_m, LOG)
plt.show()

#### Plot of `calloc()` Speed

The `calloc()` function call returns data that, when read, should return zero bytes.

In [ ]:
calloc_perf = {}
for test_run in data:
    calloc_perf[test_run] = []
    for size in sizes:
        calloc_perf[test_run].append(data[test_run]["results"][size]["t_Tc"] / 1000)
calloc_perf_m = {}
for test_run in data:
    calloc_perf_m[test_run] = []
    for size in sizes:
        calloc_perf_m[test_run].append(data[test_run]["results"][size]["t_c"] / 1000)

plot_malloc_benchmarks("calloc() speed", strsizes, calloc_perf_m, None, True)
plt.show()

### Walking Allocated Memory

The results given from the `malloc()` benchmarks and the `calloc()` benchmarks
may be returning memory very fast. Indeed, some of the data appears to be so
fast, it appears that the Operating System may not allocate the pages, and only
map it to the Operating System.

The hypotheses can be tested by walking each page returned. We know that an
Operating System must allocate a minimum of one page (this is the smallest size
of memory that the CPU MMU can manage, mapping a virtual address to a physical
address).

Often Operating Systems do *lazy* page allocation. They indicate to the MMU that
the page belongs to the process, but they do not allocate a physical page from
system memory to the process. This is done only when the process accesses the
memory (either read or write).

i.e.

- A process allocates 1GB of RAM
- The libc implementation of `malloc()` does a `mmap()` call, requesting 1GB of
  RAM.
- The Operating System updates the MMU for the current process running, by
  creating MMU Page Tables, enough for 1GB. On most Operating Systems with 4kB
  pages, this is then 262,144 pages, which map a continuous region of virtual
  memory (visible by the process).
- In *Lazy* locking, the page tables are all created in the MMU, but no pages
  from physical memory are allocated.
  - When virtual memory addresses are accessed that result in a page that is not
    yet allocated, a page fault occurs, and the Operating System will find an
    available page, resident in memory (and if the contents are in swap, it is
    copied from disk back to memory), and control is given back to the process
    without it knowing (only taking time).
  - With systems that have no swap, this has the effect that memory allocations
    usually always pass, but a crash of the program may occur at any time if
    there is no pages available.
- In *Super* locking, the pages are allocated from the free page pool
  immediately. This takes more time, but we know we will never get the page
  fault and memory accesses are fast. We know our process won't crash from out
  of memory accesses.

Thus three tests are performed that help to guage if the memory is allocated in
physical memory or not:

- BM_MallocWalkFreeBench
- BM_MallocClearFreeBench
- BM_MallocClearWalkFreeBench

The difference between the `BM_MallocWalkFreeBench` and the `BM_MallocFreeBench`
is that after memory is allocated, the value of `0` is written to the first byte
of every page. Note, we don't every byte, as then the results would be very much
dependent on the memory bandwidth of the system. Writing only a single byte we
easily measure the performance overhead if the Operating System must handle a
page fault and allocate that to the process.

Similarly, we can see if any optimisation tricks are done when clearing memory
with the `memset(p, 0, SIZE)` call, by looking at the difference between
`BM_MallocClearWalkFreeBench` and `BM_MallocClearFreeBench`.

So now:

\begin{align}
T_{MallocFreeBench} &= T_M &&&+ T_F \\
T_{MallocWalkFreeBench} &= T_M &&+ T_W &+ T_F \\
T_{MallocClearFreeBench} &= T_M &+ T_C &&+ T_F \\
T_{MallocClearWalkFreeBench} &= T_M &+ T_C &+ T_W &+ T_F \\
\end{align}

Plot the difference between the equations to get the $T_W$ walk time, and if the
values are significant, then it indicates overhead due to page faults. Some
overhead will be present due to the loops, but this is expected to be smaller
than the time required for a page fault.

where:

- $T_M$ is the call to `malloc()` as earlier.
- $T_C$ is the call to `memset(p, 0, size)`. We should be able to see if this
  causes a lot of time.
  - It appears there are optimisations that even the `memset()` is fast, without
    requiring physical pages to be allocated.
- $T_W$ is the time for the first access. If memory was not actually allocated
  by the `malloc()` or `memset()` call, this is expected to take a measurable
  amount of time. Else the access will be very fast and effectively zero.
- $T_F$ ist he call to `free()` as earlier.


In [ ]:
walk_perf = {}
for test_run in data:
    walk_perf[test_run] = []
    for size in sizes:
        walk_perf[test_run].append(data[test_run]["results"][size]["t_wm"] / 1000)

plot_malloc_benchmarks("walk after malloc() speed", strsizes, walk_perf, None, True)
plt.show()

In [ ]:
memsetwalk_perf = {}
for test_run in data:
    memsetwalk_perf[test_run] = []
    for size in sizes:
        memsetwalk_perf[test_run].append(data[test_run]["results"][size]["t_wc"] / 1000)

plot_malloc_benchmarks("walk after malloc()/memset() speed", strsizes, memsetwalk_perf, None, True)
plt.show()